In [72]:
import numpy as np
import pandas as pd
import pickle as pickle
import matplotlib.pyplot as plt
import matplotlib

def rational_to_double(rat):
    """ Takes a string of typ 4/4 or 4 and returns the corresponding float"""
    tmp = rat.split("/")
    if len(tmp) > 1:
        num = float(tmp[0])
        den = float(tmp[1])
        return num/den
    else:
        return float(tmp[0])

def string_to_list(lista):
    """ Takes a string of rationals on format 1/2 and returns a list of floats """
    splitted = lista.split()
    return [rational_to_double(x) for x in splitted]

def get_payoff_mats(payoffs, p1_n_strats):
    """ Takes a string of payoffs and return an array with the two payoff mats """
    payoff_list = [int(x) for x in payoffs.split()]
    p2_n_strats = int(len(payoff_list)/(2*p1_n_strats))

    p1_payoffs = np.array([payoff_list[i] for i in range(0,len(payoff_list)) if i % 2 == 0])
    p2_payoffs = np.array([payoff_list[i] for i in range(0,len(payoff_list)) if (i+1) % 2 == 0])

    p1_payoff_mat = np.reshape(p1_payoffs, (p1_n_strats, p2_n_strats))
    p2_payoff_mat = np.reshape(p2_payoffs, (p2_n_strats, p1_n_strats), order='F')

    return [p1_payoff_mat, p2_payoff_mat]


all_choices = pd.read_csv("../DATA_2018-11-19/choice.csv")
all_players = pd.read_csv("../DATA_2018-11-19/player.csv")
all_games = pd.read_csv("../DATA_2018-11-19/game.csv")
all_payoffs = pd.read_csv("../DATA_2018-11-19/payoff.csv")
all_pasts = pd.read_csv("../DATA_2018-11-19/past.csv")
all_gameplays = pd.read_csv("../DATA_2018-11-19/gameplay.csv")

In [73]:

all_choices["strats"] = all_choices["strats"].apply(string_to_list)
all_payoffs["payoff"] = all_payoffs["payoff"].apply(rational_to_double)
all_pasts["currentsp1"] = all_pasts["currentsp1"].apply(string_to_list)
all_pasts["currentsp2"] = all_pasts["currentsp2"].apply(string_to_list)
all_games["payoffs"] = all_games.apply(lambda row: get_payoff_mats(row["payoffs"], row["p1"]), axis=1)

In [74]:

init_strats_dict = dict()
init_p1_dict = dict()
init_p2_dict = dict()
payoffs_dict = dict()
role_dict = dict()
id_dict = dict()
actual_plays_dict = dict()
gid = 1
for gid in range(1,4):
    strats = all_choices[["strats", "playerid"]][(all_choices["round"] == 0) & (all_choices["gameid"] == gid)].copy()
    roles = all_players[["id", "role"]].copy()
    roles["playerid"] = roles["id"]
    pasts = all_pasts[["currentsp1", "currentsp2", "gameid"]][(all_pasts["round"] == 0) & (all_pasts["gameid"] == gid)].copy()

    init_p1 = all_pasts["currentsp1"][(all_pasts["round"] == 0) & (all_pasts["gameid"] == gid)].copy()
    init_p2 = all_pasts["currentsp2"][(all_pasts["round"] == 0) & (all_pasts["gameid"] == gid)].copy()

    data_mat = strats.set_index("playerid").join(roles.set_index("id"), how='outer').copy()

    # # Replace NaN with role average 
    data_mat["strats"][(data_mat["role"] == 0) & pd.isnull(data_mat["strats"])] = np.repeat(np.array(init_p1), sum((data_mat["role"] == 0) & pd.isnull(data_mat["strats"])))
    data_mat["strats"][(data_mat["role"] == 1) & pd.isnull(data_mat["strats"])] = np.repeat(np.array(init_p2), sum((data_mat["role"] == 1) & pd.isnull(data_mat["strats"])))

    plays_vec = [np.array(all_pasts["currentsp1"][all_pasts["gameid"] == gid]), np.array(all_pasts["currentsp2"][all_pasts["gameid"] == gid])]
    plays_vec[0] = np.array([np.array(v) for v in plays_vec[0]])
    plays_vec[1] = np.array([np.array(v) for v in plays_vec[1]])

    init_strats_dict[gid] = np.array(data_mat["strats"])
    init_p1_dict[gid] = init_p1
    init_p2_dict[gid] = init_p2
    payoffs_dict[gid] = all_games["payoffs"][all_games["id"]==gid]
    role_dict[gid] = np.array(data_mat["role"].copy())
    id_dict[gid] = np.array(data_mat["playerid"].copy())
    actual_plays_dict[gid] = plays_vec


/home/gustav/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gustav/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:

save_dict = {"init_strats_dict": init_strats_dict, "init_p1_dict":init_p1_dict, 
"init_p2_dict": init_p2_dict, "payoffs_dict":payoffs_dict, 
"role_dict":role_dict, "id_dict":id_dict, "actual_plays_dict":actual_plays_dict}

## Save dictionary 

with open('data_dicts.pkl', 'wb') as output:
    pickle.dump(save_dict, output, pickle.HIGHEST_PROTOCOL)


In [76]:
with open('data_dicts.pkl', 'rb') as input: 
    load_dict = pickle.load(input)
    init_strats_dict = load_dict["init_strats_dict"]
    init_p1_dict = load_dict["init_p1_dict"]
    init_p2_dict = load_dict["init_p2_dict"]
    payoffs_dict = load_dict["payoffs_dict"]
    role_dict = load_dict["role_dict"]
    id_dict = load_dict["id_dict"]
    actual_plays_dict = load_dict["actual_plays_dict"]

In [77]:
actual_plays_dict

{1: [array([[0.16666667, 0.33333333, 0.5       ],
         [0.1       , 0.62857143, 0.27142857],
         [0.35714286, 0.28571429, 0.35714286],
         [0.        , 0.5625    , 0.4375    ],
         [0.        , 0.5625    , 0.4375    ],
         [0.1       , 0.15      , 0.75      ],
         [0.        , 0.45      , 0.55      ],
         [0.04545455, 0.63636364, 0.31818182],
         [0.1       , 0.55      , 0.35      ],
         [0.08333333, 0.5       , 0.41666667],
         [0.06153846, 0.48461538, 0.45384615],
         [0.12564103, 0.39487179, 0.47948718],
         [0.08095238, 0.50952381, 0.40952381],
         [0.07555556, 0.47555556, 0.44888889],
         [0.09285714, 0.41428571, 0.49285714],
         [0.07555556, 0.54222222, 0.38222222],
         [0.12857143, 0.52142857, 0.35      ],
         [0.03333333, 0.6       , 0.36666667],
         [0.08666667, 0.55333333, 0.36      ],
         [0.08125   , 0.55      , 0.36875   ],
         [0.08095238, 0.47380952, 0.4452381 ],
         [

## Plotting stuff

In [84]:
actual_plays_dict[1][0]

array([[0.16666667, 0.33333333, 0.5       ],
       [0.1       , 0.62857143, 0.27142857],
       [0.35714286, 0.28571429, 0.35714286],
       [0.        , 0.5625    , 0.4375    ],
       [0.        , 0.5625    , 0.4375    ],
       [0.1       , 0.15      , 0.75      ],
       [0.        , 0.45      , 0.55      ],
       [0.04545455, 0.63636364, 0.31818182],
       [0.1       , 0.55      , 0.35      ],
       [0.08333333, 0.5       , 0.41666667],
       [0.06153846, 0.48461538, 0.45384615],
       [0.12564103, 0.39487179, 0.47948718],
       [0.08095238, 0.50952381, 0.40952381],
       [0.07555556, 0.47555556, 0.44888889],
       [0.09285714, 0.41428571, 0.49285714],
       [0.07555556, 0.54222222, 0.38222222],
       [0.12857143, 0.52142857, 0.35      ],
       [0.03333333, 0.6       , 0.36666667],
       [0.08666667, 0.55333333, 0.36      ],
       [0.08125   , 0.55      , 0.36875   ],
       [0.08095238, 0.47380952, 0.4452381 ],
       [0.025     , 0.54375   , 0.43125   ],
       [0.

In [88]:
rgb_cols = [(228/255, 26/255, 28/255), (55/255, 126/255, 184/255), (77/255, 175/255, 74/255), (152/255, 78/255, 163/255), (255/255, 127/255, 0/255)]
# ['rgb(228,26,28)','rgb(55,126,184)','rgb(77,175,74)','rgb(152,78,163)','rgb(255,127,0)']
matplotlib.colors.is_color_like(rgb_cols[0])


def plot_h(h, save=False):
    plt.figure(figsize=(24,8))
    for role in range(2):
        ax = plt.subplot(1,2,(role + 1))
        ax.set_ylim([-0.01,1.01])
        ax.set_yticks([0,0.25,0.5,0.75,1])
        print(h[role][1].shape)
        n_s = h[role].shape[1]
        for s in range(n_s):
            plt.plot(h[role][:,s], color=rgb_cols[s], ls="-", label="Strat "+str(s))
        ax.legend()
    if save:
        plt.savefig(save, bbox_inches="tight")
        plt.close()
    else:
        plt.show()

plot_h(actual_plays_dict[3], "game3.png")

# def plot_h_vec(h_vec, save=False):
#     plt.figure(figsize=(24,4*7))
#     for i in range(len(h_vec)):    
#         h = h_vec[i]
#         for role in range(2):
#             ax = plt.subplot(len(h_vec),2,(role + 1) + i*2)
#             ax.set_ylim([-0.01,1.01])
#             ax.set_yticks([0,0.25,0.5,0.75,1])
#             n_s = h[role].shape[1]
#             for s in range(n_s):
#                 plt.plot(h[role][:,s], color=rgb_cols[s], ls="-", label="Strat "+str(s+1))
#             ax.legend(loc="upper center", ncol=n_s, bbox_to_anchor=(0.5, 1.1), fancybox=True, fontsize="medium")
#     if save:
#         plt.savefig(save, bbox_inches='tight')
#         plt.close()
#     else:
#         plt.show()
        
    
# def get_all_pop_gid(gid, actual_plays):
#     return [actual[gid] for key,actual in actual_plays.items()]

# plot_h_vec(get_all_pop_gid(1,actual_plays_dict))

# for gid in range(1,4):
#     plot_h_vec(get_all_pop_gid(gid, actual_plays_dict), save="fig/Game_"+str(gid)+".png")



(4,)
(3,)


In [131]:
same_plays = Dict()
for pid in pd.Series.unique(all_choices.playerid):
    for gid in range(1,4):
        choices = all_choices[((all_choices.playerid == 55) & (all_choices.gameid == 2))]
        

SyntaxError: invalid syntax (<ipython-input-131-56cf4ee5ced8>, line 2)

In [136]:
pd.Series.unique(all_choices.playerid)

array([65, 53, 41, 59, 63, 38, 57, 61, 32, 60, 31, 48, 55, 34, 67, 66, 69,
       33, 56, 71, 75, 37, 73, 74, 35, 70, 36, 72, 85, 81, 86, 79, 84, 77,
       82, 42, 54, 46, 83, 87, 52, 62, 91, 39, 95])